In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/5.src'):
    shutil.rmtree('./file/5.src')

os.makedirs('./file/5.src')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/4.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['Product No'] = input_['Product No'].astype(int)
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(f'''https://gates.scene7.com/is/image/{input_.loc[a, 'Asset']}?req=set,json,UTF-8&labelkey=label&handler=s7classics7sdkJSONResponse''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                
                        if resp.status_code == 200:
                            if 'Error while processing Fvctx image' in resp.text and 'Error while processing Fvctx image' in resp.text:
                                break
                                
                            json_ = json.loads(resp.text.split('s7classics7sdkJSONResponse(')[1].split(',"");')[0])
                            if json_:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                if 'Error while processing Fvctx image' in resp.text and 'Error while processing Fvctx image' in resp.text:
                    dict_src = {}
                else:
                    dict_src, list_src = {}, [dict_['s']['n'].strip() if 's' in dict_ else dict_['set']['n'] for dict_ in json_['set']['item']] if type(json_['set']['item']) == list else [json_['set']['item']['s']['n'].strip() if 's' in json_['set']['item'] else json_['set']['item']['i']['n'].strip()]
                    for i, src in zip(range(len(list_src)), list_src):
                        dict_src[str(i)] = f'''https://gates.scene7.com/is/image/{src}'''

                    # = = = = = = = = = = = = = = =

                    if not dict_src:
                        raise

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Product No': input_.loc[a, 'Product No'],
                                         'Product Name': input_.loc[a, 'Product Name'],
                                         'Product Number': input_.loc[a, 'Product Number'],
                                         'No': input_.loc[a, 'No'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Type': input_.loc[a, 'Type'],
                                         'SKU': input_.loc[a, 'SKU'],
                                         'UPC': input_.loc[a, 'UPC'],
                                         'Title': input_.loc[a, 'Title'],
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Information': input_.loc[a, 'Json_Information'],
                                         'Part Code': input_.loc[a, 'Part Code'],
                                         'Asset': input_.loc[a, 'Asset']}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Product No']}.{input_.loc[a, 'Product Name']} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Product No', 'No'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/5.src/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Product No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-src_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/5.src'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/5.src/{file}',
              f'''./file/5.src/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 29.G34180-1616 / 16C5-16RFJX90
[剩余数量：1000] - [当前时间：17:56:42]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 27.G34175-1616 / 16C5-16RFJX45
[剩余数量：999] - [当前时间：17:56:42]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 25.G34504-0606 / 6C5-6RMIX90
[剩余数量：998] - [当前时间：17:56:42]

[状态：ok，尝试次数：2] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 31.G34206-0606 / 6C5-6RFSX90L
[剩余数量：997] - [当前时间：17:56:42]

[状态：ok，尝试次数：1] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 30.G34181-1616 / 16C5-16RFJX90L
[剩余数量：996] - [当前时间：17:56:43]

[状态：ok，尝试次数：4] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 32.G34100-0606B / 6C5-6RMPB
[剩余数量：995] - [当前时间：17:56:43]

[状态：ok，尝试次数：4] - 100.00% - crawler_2 > 71.D.O.T. Type A1 Field Attachable Couplings > 33.G34100-1008B / 10C5-8RMPB
[剩余数量：9

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3766.78it/s]

Done ~
